# Векторизация, Numba

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html
* https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types
* https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html
* https://numba.pydata.org/numba-doc/latest/user/vectorize.html


## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`. Ускорьте вычисления при помощи numba

In [ ]:
A = np.random.randint(0, 1000, size=(1000000,))

In [ ]:
def f1(A):
    acc, cnt = 0, 0
    for x in A:
        acc += x + 100
        cnt += 1
    return acc / cnt

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Создайте массив, заполненный случайными целыми неотрицательными числами и примените функцию к каждому элементу массива.

3. Приведите все слова из столбца key к верхнему регистру

In [ ]:
import pandas as pd
import string
import numpy as np

def create_df(allow_nan=False, N=2_000_000):
    df = pd.DataFrame(np.random.randint(0, 10, (N, 4)), columns=[f"col{i}" for i in range(4)])
    names = ["Apple",  "Banana",  "Apricot",  "Atemoya",  "Avocados",  "Blueberry",  "Blackcurrant",  "Ackee",  "Cranberry",  "Cantaloupe",  "Cherry",  "Black sapote/Chocolate pudding fruit",  "Dragonrfruit",  "Dates",  "Cherimoya",  "Buddha’s hand fruit",  "Finger Lime",  "Fig",  "Coconut",]
    if allow_nan:
        names.append(None)
    df["key"] = np.random.choice(names, N, replace=True)
    return df

In [ ]:
df = create_df(allow_nan=False, N=2_000_000)

4\. Для каждой строки рассчитайте разность между значениями col0 и col1, если в столбце col3 стоит четное число, и разность между col0 и col2 в противном случае.

In [ ]:
df = create_df(allow_nan=False, N=500_000).select_dtypes('number')

## Лабораторная работа 2

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [24]:
import pandas as pd
import numpy as np
import numba
import json
import sklearn.metrics

В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

In [2]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')

## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

In [3]:
def mape(actual , pred):
    buf = []
    for i in range(len(actual)):
        buf.append(abs(actual[i] - pred[i]) / actual[i])
            
    return(sum(buf)/len(buf))

In [4]:
def mape_numpy(actual , pred): 
    return np.mean(np.abs((actual - pred) / actual)) 

In [30]:
@numba.jit(nopython = True)
def mape_numba(k1, k2):
    k3 = 0
    for i in range (len(k1)):
        k3 = k3 + (abs(k1[i] - k2[i])/k1[i])
    k4 = k3/len(k1)
    return k4

№1\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

In [6]:
with open('rating_predictions.json') as f:
    values = json.load(f)

A_list = []
F_list = []

for i in values:
    A_list.append(i.get('rating'))
    F_list.append(i.get('prediction'))

In [25]:
sklearn.metrics.mean_absolute_percentage_error(A_list, F_list)

0.13325265503991449

In [7]:
%%time
mape(A_list, F_list)

Wall time: 31.2 ms


0.13325265503992637

№1\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [8]:
%%time
A_list, F_list = np.array(A_list), np.array(F_list)
mape_numpy(A_list, F_list)

Wall time: 20.1 ms


0.13325265503991449

№1\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`.

In [33]:
%%time
k1 = numba.typed.List(A_list)
k2 = numba.typed.List(F_list)
mape_numba(k1, k2)

Wall time: 450 ms


0.13325265503992637

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



№2\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [10]:
ids = reviews["user_id"].unique()
ids

array([     21752,     431813,     400708, ...,    1270706,    2282344,
       2000242659], dtype=int64)

In [11]:
import math

In [12]:
def is_pretty(n: int) -> bool:
    x = [(n//(10**i))%10 for i in range(math.ceil(math.log(n, 10))-1, -1, -1)]
    if x[0] == x[len(x)-1]:
        return True
    else:
        return False

In [13]:
is_pretty(ids[0])

True

№2\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

№2\.3. При помощи декоратора `numpy.vectorize` создайте векторизованную версию функции `is_pretty`. Корректно использовав эту векторизованную функцию, посчитайте количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [16]:
is_pretty_vect = np.vectorize(is_pretty)

In [26]:
%%time
ans = is_pretty_vect(ids)
ans

Wall time: 124 ms


array([ True, False, False, ..., False, False, False])

In [18]:
np.count_nonzero(ans == True)

4389

№2\.4. При помощи декоратора `numba.vectorize` создайте векторизованную версию функции `is_pretty`. Корректно использовав эту векторизованную функцию, посчитайте количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [23]:
from numba import vectorize, bool_, int64

In [20]:
@vectorize([bool_(int64)])
def is_pretty(n: int) -> bool:
    x = [(n//(10**i))%10 for i in range(math.ceil(math.log(n, 10))-1, -1, -1)]
    if x[0] == x[len(x)-1]:
        return True
    else:
        return False

C:\Users\vladi\AppData\Local\Temp\ipykernel_18168\3057749514.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "is_pretty" failed type inference due to: No implementation of function Function(<built-in function log>) found for signature:
 
 >>> log(int64, Literal[int](10))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Type Restricted Function in function 'log': File: unknown: Line unknown.
    With argument(s): '(int64, int64)':
   No match for registered cases:
    * (int64,) -> float64
    * (uint64,) -> float64
    * (float32,) -> float32
    * (float64,) -> float64

During: resolving callee type: Function(<built-in function log>)
During: typing of call at C:\Users\vladi\AppData\Local\Temp\ipykernel_18168\3057749514.py (3)


File "AppData\Local\Temp\ipykernel_18168\3057749514.py", line 3:
<source missing, REPL/exec in use?>

  @vectorize([bool_(int64)])
C:\Users\vladi\anaconda3\lib\site-pack

In [29]:
%%time
nans = is_pretty(ids)
nans

Wall time: 134 ms


array([ True, False, False, ..., False, False, False])

In [28]:
np.count_nonzero(nans == True)

4389